In [1]:
import bibtexparser
from bibtexparser.bparser import BibTexParser 


def test_bibtexparser():
    """Test bibtexparser library
    https://bibtexparser.readthedocs.org/en/latest/tutorial.html
    """
    with open('wjpubs.bib') as bibtex_file:
        bib_db = bibtexparser.load(bibtex_file)
    print bib_db.entries



In [2]:
#test_bibtexparser()

In [3]:
import bibtexparser.customization as bcus


In [4]:
# Let's define a function to customize our entries.
# It takes a record and return this record.
def customizations(record):
    """Use some functions delivered by the library

    :param record: a record
    :returns: -- customized record
    """
    record = bcus.type(record)
    record = bcus.author(record)
    record = bcus.editor(record)
    #record = bcus.journal(record)
    record = bcus.keyword(record)
    #record = bcus.link(record)
    record = bcus.page_double_hyphen(record)
    record = bcus.doi(record)
    return record


In [5]:
import pprint 
fname = 'wjpubs.bib'
with open(fname) as bibtex_file:
    parser = BibTexParser()
    parser.customization = customizations
    bib_db = bibtexparser.load(bibtex_file, parser=parser)
    #print bib_db.entries
    pprint.pprint(bib_db.entries)

[{u'author': [u'Jitkrittum, Wittawat',
              u'Gretton, Arthur',
              u'Heess, Nicolas',
              u'Eslami, S. M. Ali',
              u'Lakshminarayanan, Balaji',
              u'Sejdinovic, Dino',
              u'Szab\xf3, Zolt\xe1n'],
  'id': 'jitkrittum_kernel-based_2015',
  u'journal': u'arXiv:1503.02551',
  u'link': u'http://arxiv.org/abs/1503.02551',
  u'title': u'Kernel-{Based} {Just}-{In}-{Time} {Learning} for {Passing} {Expectation} {Propagation} {Messages}',
  'type': u'article',
  u'wj_http': u'http://arxiv.org/abs/1503.02551',
  u'year': u'2015'},
 {u'author': [u'Park, Mijung', u'Jitkrittum, Wittawat', u'Sejdinovic, Dino'],
  'id': 'part_k2abc_2015_arxiv',
  u'journal': u'{arXiv}:1502.02558',
  u'link': u'http://arxiv.org/abs/1502.02558',
  u'title': u'{K2-ABC}: Approximate {B}ayesian Computation with Infinite Dimensional Summary Statistics via Kernel Embeddings',
  'type': u'article',
  u'wj_http': u'http://arxiv.org/abs/1502.02558',
  u'year': u'2015

In [6]:
def format_author(au_list, au_func=None):
    """au_func is a function taking in first name last name and return 
    an HTML formatted text.
    Format the author list into plain text"""
    
    fl_list = []
    for a in au_list:
        lf = [x.strip() for x in a.split(',')]
        fn = '%s %s'%(lf[1], lf[0])
        formatted = au_func(fn) if au_func is not None else fn
        fl_list.append(formatted)
    s = ', '.join(fl_list[:-1]) + ', and ' + fl_list[-1]
    return s

 
au_list = bib_db.entries[0]['author']
print format_author(au_list)

Wittawat Jitkrittum, Arthur Gretton, Nicolas Heess, S. M. Ali Eslami, Balaji Lakshminarayanan, Dino Sejdinovic, and Zoltán Szabó


In [7]:
from bibtexparser.bibdatabase import BibDatabase

def id2bibsrc(fname, ignore_key_list=None):
    src = {}
    with open(fname) as bibtext_file:
        raw_db = bibtexparser.load(bibtext_file)
        for en in raw_db.entries:
            tmp_db = BibDatabase()
                
            if ignore_key_list is not None:
                for k in ignore_key_list:
                    en.pop(k, None)
            
            tmp_db.entries = [en]
            en_src = bibtexparser.dumps(tmp_db)
            src[en['id']] = en_src
    return src

src = id2bibsrc(fname)
src['Jitkrittum2009']

u"@inproceedings{Jitkrittum2009,\n author = {Wittawat Jitkrittum and  Choochart Haruechaiyasak and Thanaruk Theeramunkong},\n booktitle = {Proceedings of the 2009 Workshop on Knowledge and Reasoning for Answering\nQuestions},\n link = {http://dl.acm.org/citation.cfm?id=1697288.1697291},\n pages = {11--14},\n publisher = {Association for Computational Linguistics},\n series = {KRAQ '09},\n title = {{QAST}: Question Answering System for {Thai} Wikipedia},\n wj_http = {http://dl.acm.org/citation.cfm?id=1697288.1697291},\n year = {2009}\n}\n\n"

In [8]:
def clean_value(st):
    return st.replace('{', '').replace('}', '').replace('\&', '&')

def format_key(key, entry, map_func):
    form = map_func[key](entry[key]) if key in map_func else entry[key]
    form = clean_value(form)
    return form
                        
def format_bib_db(bib_db, map_func):
    """
    bib_db: a list of dictionaries from bibtexparser.load(..)
    map_func: a dict mapping from bibtex keys to functions to format the values
    associated with the keys"""
   
    html = ''
    for entry in bib_db:
        html += format_key('author', entry, map_func) + '.\n'
        html += format_key('title', entry, map_func) + '\n'
        #pprint.pprint(entry)
        if 'journal' in entry:
            html += format_key('journal', entry, map_func)
        elif 'booktitle' in entry:
            html += format_key('booktitle', entry, map_func)    
        html += ', '
        if 'year' in entry:
            html += format_key('year', entry, map_func)    
        html += '. '
        if 'link' in entry:
            html += 'URL: ' + format_key('link', entry, map_func)    
            
        html += '\n\n'
    return html
    

def author_func(author):
    a = author.strip()
    if a=='Wittawat Jitkrittum':
        return '<b>Wittawat Jitkrittum</b>'
    
    def link_author(text, href):
        return '<a href="%s">%s</a>'%(href, text)
    M = dict([
        ('Arthur Gretton', 'http://www.gatsby.ucl.ac.uk/~gretton/'),
        ('Nicolas Heess', 'http://homepages.inf.ed.ac.uk/s0677090/'),
        ('S. M. Ali Eslami', 'http://arkitus.com/'),
        ('Balaji Lakshminarayanan', 'http://www.gatsby.ucl.ac.uk/~balaji/'),
        ('Dino Sejdinovic', 'http://www.stats.ox.ac.uk/~sejdinov/'),
        (u'Zoltán Szabó', 'http://www.gatsby.ucl.ac.uk/~szabo/'),
        ('Mijung Park', 'http://www.gatsby.ucl.ac.uk/~mijung/'),
        ('Makoto Yamada', 'http://www.makotoyamada-ml.com/'),
        ('Leonid Sigal', 'http://cs.brown.edu/~ls/'),
        ('Eric P. Xing', 'http://www.cs.cmu.edu/~epxing/'),
        ('Masashi Sugiyama', 'http://www.ms.k.u-tokyo.ac.jp/index.html'),
        ('Hirotaka Hachiya', 'http://sugiyama-www.cs.titech.ac.jp/~hachiya/'),
        ('Gang Niu', 'http://sugiyama-www.cs.titech.ac.jp/~gang/'),
        ('Bo Dai', 'https://sites.google.com/site/daibohr/'), 

    ]);
    if a in M:
        return link_author(a, M[a])
    else:
        return a
    
map_func = {'author': format_author}
print format_bib_db(bib_db.entries, map_func)

Wittawat Jitkrittum, Arthur Gretton, Nicolas Heess, S. M. Ali Eslami, Balaji Lakshminarayanan, Dino Sejdinovic, and Zoltán Szabó.
Kernel-Based Just-In-Time Learning for Passing Expectation Propagation Messages
arXiv:1503.02551, 2015. URL: http://arxiv.org/abs/1503.02551

Mijung Park, Wittawat Jitkrittum, and Dino Sejdinovic.
K2-ABC: Approximate Bayesian Computation with Infinite Dimensional Summary Statistics via Kernel Embeddings
arXiv:1502.02558, 2015. URL: http://arxiv.org/abs/1502.02558

Wittawat Jitkrittum, Arthur Gretton, and Nicolas Heess.
Passing Expectation Propagation Messages with Kernel Methods
arXiv:1501.00375, 2015. URL: http://arxiv.org/abs/1501.00375

Makoto Yamada, Wittawat Jitkrittum, Leonid Sigal, Eric P. Xing, and Masashi Sugiyama.
High-Dimensional Feature Selection by Feature-Wise Kernelized
Lasso
Neural Computation, 2014. URL: http://www.mitpressjournals.org/doi/abs/10.1162/NECO_a_00537#.U9O7Idtsylg

Wittawat Jitkrittum, Hirotaka Hachiya, and Masashi Sugiyama.
Fea

In [10]:

def format_html_bib_db(bib_db, map_func, map_src):
    """
    bib_db: a list of dictionaries from bibtexparser.load(..)
    map_func: a dict mapping from bibtex keys to functions to format the values
    associated with the keys. 
    map_src: a dict from bib key to bib source of the entry.
    
    Format the bib database into html. """
         
    html = '<div class="publications"> \n\n'
    for entry in bib_db:
        html += '<div class="paper"> \n'
        html += '<div class="paper_title">%s</div>\n'%( format_key('title', entry, map_func) )
        
        html += '<div class="author_list">%s</div>\n'%( format_key('author', entry, map_func) )
        #pprint.pprint(entry)
        place = ''
        if 'journal' in entry:
            place = format_key('journal', entry, map_func)
        elif 'booktitle' in entry:
            place = format_key('booktitle', entry, map_func)    
        html += '<span class="submit_to">%s</span>, '%place
        if 'year' in entry:
            html += format_key('year', entry, map_func)    
        html += '. \n'
        #if 'link' in entry:
        #    html += 'URL: ' + format_key('link', entry, map_func)  + '\n'
        html += '<div class="links">Link: \n'
        if 'wj_http' in entry:
            html += '<a class="wj_http" href="%s">http</a>\n'%( entry['wj_http'] )
        if 'wj_pdf' in entry:
            html += '<a class="wj_pdf" href="%s">pdf</a>\n'%( entry['wj_pdf'] )
        # bib link 
        html += '<a class="biblink">bib</a> \n'
        html += '</div> <!-- end links -->\n'
        # bib source. Can use jquery to make a sliding effect.
        html += '<pre class="bibsrc">%s</pre><!-- end bib source -->\n' % (map_src[entry['id']])
        
        # end paper
        html += '</div>'
        
        html += '\n\n'
    
    html += '</div> \n'
    return html
    
ignore_keys = ['wj_http', 'wj_pdf']
src = id2bibsrc(fname, ignore_keys)
map_func = {'author': lambda au_list: format_author(au_list, author_func)}
print format_html_bib_db(bib_db.entries, map_func, src)


<div class="publications"> 

<div class="paper"> 
<div class="paper_title">Kernel-Based Just-In-Time Learning for Passing Expectation Propagation Messages</div>
<div class="author_list"><b>Wittawat Jitkrittum</b>, <a href="http://www.gatsby.ucl.ac.uk/~gretton/">Arthur Gretton</a>, <a href="http://homepages.inf.ed.ac.uk/s0677090/">Nicolas Heess</a>, <a href="http://arkitus.com/">S. M. Ali Eslami</a>, <a href="http://www.gatsby.ucl.ac.uk/~balaji/">Balaji Lakshminarayanan</a>, <a href="http://www.stats.ox.ac.uk/~sejdinov/">Dino Sejdinovic</a>, and <a href="http://www.gatsby.ucl.ac.uk/~szabo/">Zoltán Szabó</a></div>
<span class="submit_to">arXiv:1503.02551</span>, 2015. 
<div class="links">Link: 
<a class="wj_http" href="http://arxiv.org/abs/1503.02551">http</a>
<a class="biblink">bib</a> 
</div> <!-- end links -->
<pre class="bibsrc">@article{jitkrittum_kernel-based_2015,
 author = {Jitkrittum, Wittawat and Gretton, Arthur and Heess, Nicolas and Eslami, S. M. Ali and Lakshminarayanan, Bal